# Lab SQL 9
In this lab we will find the customers who were active in consecutive months of May and June. Follow the steps to complete the analysis.


## Create a table `rentals_may` to store the data from rental table with information for the month of May.

In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

import getpass  # to get the password without showing the input
password = getpass.getpass()

connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

········


In [2]:
query = '''
CREATE TABLE rentals_may(
rental_id INT UNIQUE NOT NULL, 
rental_date DATETIME DEFAULT NULL,
inventory_id MEDIUMINT UNSIGNED DEFAULT NULL,
customer_id SMALLINT UNSIGNED DEFAULT NULL,
return_date DATETIME DEFAULT NULL,
staff_id TINYINT UNSIGNED DEFAULT NULL,
last_update TIMESTAMP DEFAULT NULL,
CONSTRAINT PRIMARY KEY (rental_id)
)'''
rentals_may = pd.read_sql_query(query, engine)
rentals_may

OperationalError: (pymysql.err.OperationalError) (1050, "Table 'rentals_may' already exists")
[SQL: 
CREATE TABLE rentals_may(
rental_id INT UNIQUE NOT NULL, 
rental_date DATETIME DEFAULT NULL,
inventory_id MEDIUMINT UNSIGNED DEFAULT NULL,
customer_id SMALLINT UNSIGNED DEFAULT NULL,
return_date DATETIME DEFAULT NULL,
staff_id TINYINT UNSIGNED DEFAULT NULL,
last_update TIMESTAMP DEFAULT NULL,
CONSTRAINT PRIMARY KEY (rental_id)
)]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

## Insert values in the table `rentals_may` using the table rental, filtering values only for the month of May.
 

In [3]:
query_1 = ''' INSERT INTO rentals_may
SELECT * FROM rental 
WHERE MONTH(rental_date) = 05;

'''

rentals_may = pd.read_sql_query(query_1, engine)
rentals_may


IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '1' for key 'rentals_may.PRIMARY'")
[SQL:  INSERT INTO rentals_may
SELECT * FROM rental 
WHERE MONTH(rental_date) = 05;

]
(Background on this error at: http://sqlalche.me/e/13/gkpj)

## Create a table `rentals_june` to store the data from rental table with information for the month of June.


In [4]:
query_2 = '''
CREATE TABLE rentals_june(
rental_id INT UNIQUE NOT NULL, 
rental_date DATETIME DEFAULT NULL,
inventory_id MEDIUMINT UNSIGNED DEFAULT NULL,
customer_id SMALLINT UNSIGNED DEFAULT NULL,
return_date DATETIME DEFAULT NULL,
staff_id TINYINT UNSIGNED DEFAULT NULL,
last_update TIMESTAMP DEFAULT NULL,
CONSTRAINT PRIMARY KEY (rental_id)
)'''
rentals_june = pd.read_sql_query(query_2, engine)
rentals_june

OperationalError: (pymysql.err.OperationalError) (1050, "Table 'rentals_june' already exists")
[SQL: 
CREATE TABLE rentals_june(
rental_id INT UNIQUE NOT NULL, 
rental_date DATETIME DEFAULT NULL,
inventory_id MEDIUMINT UNSIGNED DEFAULT NULL,
customer_id SMALLINT UNSIGNED DEFAULT NULL,
return_date DATETIME DEFAULT NULL,
staff_id TINYINT UNSIGNED DEFAULT NULL,
last_update TIMESTAMP DEFAULT NULL,
CONSTRAINT PRIMARY KEY (rental_id)
)]
(Background on this error at: http://sqlalche.me/e/13/e3q8)

## Insert values in the table `rentals_june` using the table rental, filtering values only for the month of June.
 

In [5]:
query_3 = ''' INSERT INTO rentals_june
 SELECT *
 FROM rental
 WHERE MONTH(rental_date) = 06;

'''

rentals_june = pd.read_sql_query(query_3, engine)

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '1158' for key 'rentals_june.PRIMARY'")
[SQL:  INSERT INTO rentals_june
 SELECT *
 FROM rental
 WHERE MONTH(rental_date) = 06;

]
(Background on this error at: http://sqlalche.me/e/13/gkpj)

## Check the number of rentals for each customer for May.


In [6]:
query_4 = ''' SELECT customer_id, COUNT(rental_id) AS rentals_may FROM rentals_may
GROUP BY customer_id
ORDER BY COUNT(rental_id) desc;

'''

rentals_may_customer = pd.read_sql_query(query_4, engine)
pd.DataFrame(rentals_may_customer) 

,customer_id,rentals_may
0,197,8
1,506,7
2,109,7
3,269,6
4,239,6
...,...,...
515,431,1
516,351,1
517,10,1
518,136,1


## Check the number of rentals for each customer for June.
 

In [7]:
query_5 = ''' SELECT customer_id, COUNT(rental_id) AS rentals_june FROM rentals_june
GROUP BY customer_id
ORDER BY COUNT(rental_id) desc;

'''

rentals_june_customer = pd.read_sql_query(query_5, engine)
pd.DataFrame(rentals_june_customer) 

,customer_id,rentals_june
0,31,11
1,454,10
2,329,9
3,295,9
4,561,9
...,...,...
585,425,1
586,412,1
587,335,1
588,22,1


## Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:

  - Check the number of rentals for each customer for May
  - Check the number of rentals for each customer for June

    **Hint**: You can store the results from the two queries in two separate dataframes.
 

In [ ]:
# I think I already did this. 

## Write a function that checks if customer borrowed more or less films in the month of June as compared to May.
 
 **Hint**: For this part, you can create a join between the two dataframes created before, using the merge function available for pandas dataframes. Here is a link to the documentation for the [merge function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html).

In [8]:
df_merge = pd.merge(rentals_may_customer, rentals_june_customer, on = "customer_id")
df_merge

,customer_id,rentals_may,rentals_june
0,197,8,8
1,506,7,5
2,109,7,5
3,269,6,3
4,239,6,5
...,...,...,...
507,234,1,8
508,431,1,7
509,351,1,3
510,10,1,5


In [9]:
def month_rentals(x):
    if x["rentals_may"] > x["rentals_june"]:
        return 'rent more in may'
    if x["rentals_may"] < x["rentals_june"]:
        return 'rent more in june'
    else:
        return 'draw'
    
df_merge['month_rentals'] = df_merge.apply(month_rentals, axis=1)
df_merge

,customer_id,rentals_may,rentals_june,month_rentals
0,197,8,8,draw
1,506,7,5,rent more in may
2,109,7,5,rent more in may
3,269,6,3,rent more in may
4,239,6,5,rent more in may
...,...,...,...,...
507,234,1,8,rent more in june
508,431,1,7,rent more in june
509,351,1,3,rent more in june
510,10,1,5,rent more in june
